In the exercise below, all the code is given to extract HOG features and train a linear SVM. There is no right or wrong answer, but the mission is to play with the parameters **colorspace, orient, pix_per_cell, cell_per_block, and hog_channel** to get a feel for what combination of parameters give the best results.

Note: hog_channel can take values of 0, 1, 2, or "ALL", meaning that I extract HOG features from the first, second, third, or all color channels respectively. 

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import glob
import time
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from skimage.feature import hog

In [99]:
def get_hog_features(img,orient,pix_per_cell,cell_per_block,vis=False,feature_vec=True):
        
        if vis==True:
            features,hog_image=hog(img,orientations=orient,
                                     pixels_per_cell=(pix_per_cell,pix_per_cell),
                                     cells_per_block=(cell_per_block,cell_per_block),
                                     block_norm="L2-Hys",
                                     transform_sqrt=True,
                                     visualise=vis,
                                     feature_vector=feature_vec)
            return features,hog_image
        else:
            features=hog(img,orientations=orient,
                                     pixels_per_cell=(pix_per_cell,pix_per_cell),
                                     cells_per_block=(cell_per_block,cell_per_block),
                                     block_norm="L2-Hys",
                                     transform_sqrt=True,
                                     visualise=vis,
                                     feature_vector=feature_vec)
            return features
    

In [114]:
def extract_features(imgs,cspace='RGB',orient=9,pix_per_cell=8,cell_per_block=2,hog_channel=0):
    features=[]
    for file in imgs:
        image=mpimg.imread(file)
        if cspace!='RGB':
            if cspace=='HSV':
                feature_image=cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
            if cspace=='LUV':
                feature_image=cv2.cvtColor(image,cv2.COLOR_RGB2LUV)
            if cspace=='HLS':
                feature_image=cv2.cvtColor(image,cv2.COLOR_RGB2HLS)                
            if cspace=='YUV':
                feature_image=cv2.cvtColor(image,cv2.COLOR_RGB2YUV) 
            if cspace=='YCrCb':
                feature_image=cv2.cvtColor(image,cv2.COLOR_RGB2YCrCb)
        else:
            feature_image=np.copy(image)
        
        
        if hog_channel=="ALL":
            hog_features= []
            for channel in range(feature_image.shape[2]):
                hog_features.append(get_hog_features(feature_image[:,:,channel],
                                                    orient,pix_per_cell,cell_per_block,
                                                    vis=False,feature_vec=True))
            hog_features=np.ravel(hog_features)
        else:
            hog_features=get_hog_features(feature_image[:,:,hog_channel],
                                                    orient,pix_per_cell,cell_per_block,
                                                    vis=False,feature_vec=True)
        features.append(hog_features)
        
    return features

In [115]:
def get_dataset(CarDirectory,NonCarDirectory):
    CarImages=[]
    NonCarImages=[]
    CarImages=glob.glob(CarDirectory,recursive=True)
    NonCarImages=glob.glob(NonCarDirectory,recursive=True)
    
    data_dict={}
    
    data_dict['CarImages']=CarImages
    data_dict['NonCarImages']=NonCarImages
    
    # Define a key in data_dict "n_cars" and store the number of car images
    data_dict["n_cars"] = len(CarImages)
    # Define a key "n_notcars" and store the number of notcar images
    data_dict["n_notcars"] = len(NonCarImages)
    # Read in a test image, either car or notcar
    example_img = mpimg.imread(CarImages[0])
    # Define a key "image_shape" and store the test image shape 3-tuple
    data_dict["image_shape"] = example_img.shape
    # Define a key "data_type" and store the data type of the test image.
    data_dict["data_type"] = example_img.dtype
    return data_dict

In [116]:
CarDirectory='../dataset/vehicles_smallset/*/*.jpeg'
NonCarDirectory='../dataset/non-vehicles_smallset/*/*.jpeg'
data_dict=get_dataset(CarDirectory,NonCarDirectory)

In [128]:
print('Your function returned a count of', 
      data_dict["n_cars"], ' cars and', 
      data_dict["n_notcars"], ' non-cars')

print('of size: ',data_dict["image_shape"], ' and data type:', 
      data_dict["data_type"])

Your function returned a count of 1196  cars and 1125  non-cars
of size:  (64, 64, 3)  and data type: uint8


In [155]:
colorspace = 'HLS' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9
pix_per_cell = 8
cell_per_block = 2
hog_channel = "ALL" # Can be 0, 1, 2, or "ALL"

In [156]:
t=time.time()
car_features=extract_features(data_dict['CarImages'],cspace=colorspace,
                              orient=orient,pix_per_cell=pix_per_cell,
                             cell_per_block=cell_per_block,
                             hog_channel=hog_channel)
print(round(time.time()-t, 2), 'Seconds to extract HOG features...')


11.18 Seconds to extract HOG features...


In [157]:
t=time.time()
Noncar_features=extract_features(data_dict['NonCarImages'],cspace=colorspace,
                              orient=orient,pix_per_cell=pix_per_cell,
                             cell_per_block=cell_per_block,
                             hog_channel=hog_channel)
print(round(time.time()-t, 2), 'Seconds to extract HOG features...')

10.54 Seconds to extract HOG features...


In [158]:
# Create an array stack of feature vectors
X=np.vstack((car_features,Noncar_features)).astype(np.float64)

In [159]:
# Define the labels vector
y=np.hstack((np.ones(len(car_features)),np.zeros(len(Noncar_features))))

In [160]:
print(X.shape)
print(y.shape)

(2321, 5292)
(2321,)


In [161]:
rand_state=np.random.randint(0,100)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=rand_state)


In [162]:
X_scaler=StandardScaler().fit(X_train)
X_train=X_scaler.transform(X_train)
X_test=X_scaler.transform(X_test)

In [163]:
svc=LinearSVC()
t=time.time()
svc.fit(X_train,y_train)
print(round(time.time()-t,2),'Second to train SVC..')


4.06 Second to train SVC..


In [164]:
print('Test Accuracy of SVC',round(svc.score(X_test,y_test),5))

Test Accuracy of SVC 0.92473


In [165]:
# Check the prediction time for a single sample
t=time.time()
n_predict = 10
print('My SVC predicts:', svc.predict(X_test[0:n_predict]))
print('For those',n_predict,'Labels:',y_test[0:n_predict])
print(round(time.time()-t,5),'Second to predict SVC..')


My SVC predicts: [0. 0. 0. 1. 0. 0. 1. 0. 1. 1.]
For those 10 Labels: [0. 0. 0. 1. 0. 0. 1. 0. 1. 0.]
0.00599 Second to predict SVC..
